In [2]:
# Cоздание класса ElGam
from random import randint
from sage.crypto.util import bin_to_ascii

class ElGam:
    
    def __init__(self, p: int):
        self.p = p
        self.phi_p = euler_phi(p)
        self.g = randint(2, p - 1)
        while pow(self.g, self.phi_p, self.p) != 1:
            self.g = randint(2, p - 1)
    def ascii_to_integer(self, text: str):
        """
        param: text -- исходный текст
        return: number -- числовое представление текста

        """
        number, r = 0, len(text)
        for i in range(r):
            number += (ord(text[i]) << (8 * i))
        return number

    def er(self, num: int):
        """
        расширитель строки. до числа бинарных символов кратных 8.
        вход num - сумма символов в ASCII 
        выход string - строка бинарных символов длиной кратной 8.
        """
        string = str(bin(num)).replace('0b', '')
        counter = len(string) % 8
        for i in range(8 - counter):
            string = '0' + string
        return string
    
    def generation(self):
        
        self.x = randint(2, self.p - 2)
        self.y = int(pow(self.g, self.x, self.p))
        return self.y, self.g, self.p, self.x
    
    def encryption(self, m = "test", p = 0, y = 0, g = 0):
        if p == 0:
            p = self.p
        if y == 0:
            y = self.y
        k = randint(2, p - 2)
        while gcd(k, p-1) != 1: 
            k = randint(2, p - 2)
        a = int(pow(g, k, p))
        M = self.ascii_to_integer(m)
        b = (int(pow(y, k, p)) * M ) % p
        return a, b
    
    def decryption(self, a, b, x = 0, p = 0):
        if p == 0:
            p = self.p
        if x == 0:
            x = self.x
        M = (b * int(pow(a, p - 1 - x, p)))% p 
        self.m = bin_to_ascii(self.er(M))[::-1]
        return self.m 

In [5]:
# Cоздание класса RSA
from random import randint
from sage.crypto.util import bin_to_ascii


class RSA:
    
    def __init__ (self, p: int,q: int):
        """
        Инициализация шифра
        
        """
        self.N = q * p
        self.phi_N = euler_phi(self.N)
    
    def ascii_to_integer(self, text: str):
        """
        param: text -- исходный текст
        return: number -- числовое представление текста

        """
        number, r = 0, len(text)
        for i in range(r):
            number += (ord(text[i]) << (8 * i))
        return number

    def er(self, num: int):
        """
        расширитель строки. до числа бинарных символов кратных 8.
        вход num - сумма символов в ASCII 
        выход string - строка бинарных символов длиной кратной 8.
        """
        string = str(bin(num)).replace('0b', '')
        counter = len(string) % 8
        for i in range(8 - counter):
            string = '0' + string
        return string
    
    def generation(self):
        """ 
        Создание ex, dx для Nx
        
        """
        self.d = 0
        self.e = N
        while self.d < 1 / 3 * float(real_nth_root(self.N, 4)):
            self.e = randint(2, self.phi_N - 1)
            try:
                self.d = inverse_mod(self.e, self.phi_N)
            except ZeroDivisionError:
                self.d = 0
        return self.e, self.d
    
    def encryption(self, e, m, N = 0):
        """
        
        """
        if N == 0:
            N = self.N
        #x = self.ascii_to_integer(m)
        c = int(pow(m, e, N) )
        return c
    
    def decryption(self, d, ciphertext, N = 0):
        if N == 0:
            N = self.N
        aux = pow(ciphertext, d, N)
        #self.m = bin_to_ascii(self.er(aux))[::-1]
        return aux
    
    def sign(self, d, m, N = 0):
        if N == 0:
            N = self.N
        #x = self.ascii_to_integer(m)
        self.sig = int(pow(m, d, N))
        return self.sig, m
    
    def sign_check(self, e, sig,  N = 0):
        if N == 0:
            N = self.N
        aux = pow(sig[0], e, N)
        #m_shtrih = bin_to_ascii(self.er(aux))[::-1]
        return sig[1] ==  aux
    
    def EncSign_encryption(self, e2, N2, d1, N1, m):
        c = self.encryption(e2, m, N2)
        sign = self.sign(d1, c, N1)
        return sign
    
    def EncSign_decryption(self, d2, N2, e1, N1, sign):
        if self.sign_check(e1, sign, N1) == True:
            m = self.decryption(d2, sign[1], N2)
            return m
            

In [6]:
def Elgam_EncSign(elgambob, rsaalisa, PK_bob_eg, SK_alisa_RSA,  mes = 'test'):
    yb, gb, pb = PK_bob_eg 
    da, Na = SK_alisa_RSA
    ab, bb = elgamalisa.encryption(m = mes, p = pb, y = yb, g = gb)
    sign_for_bob_ab = rsaalisa.sign(da, ab, Na)
    sign_for_bob_bb = rsaalisa.sign(da, bb, Na)
    return sign_for_bob_ab, sign_for_bob_bb

def Elgam_encSign_dec(elgambob, rsabob, SK_bob_eg, PK_alisa_RSA,  ab, bb):
    xb, pb = SK_bob_eg
    ea, Na = PK_alisa_RSA
    if rsabob.sign_check(ea, ab, Na ) == True and rsabob.sign_check(ea, bb, Na ) == True:
        mes = elgambob.decryption(a = ab[1], b = bb[1], x = xb, p = pb)
        return mes
    else:
        return None


In [7]:
import time

elgambob = ElGam(random_prime(2 ^120))
yb, gb, pb, xb = elgambob.generation()

a, b = elgambob.encryption(m = 'test', p = pb, y=yb)
mes = elgambob.decryption(a = a,b = b,p = pb, x = xb)

PK_bob_eg = []
SK_bob_eg = []
                 
PK_bob_eg.append(yb)
PK_bob_eg.append(gb)
PK_bob_eg.append(pb)
                                 
SK_bob_eg.append(xb)
SK_bob_eg.append(pb)
                 
                 
                 
PK_alisa_eg = []
SK_alisa_eg = []
                 
                 
elgamalisa = ElGam(random_prime(2 ^91, 2 ^ 92))
ya, ga, pa, xa = elgamalisa.generation()
                 
PK_alisa_eg.append(ya)
PK_alisa_eg.append(ga)
PK_alisa_eg.append(pa)
                                 
SK_alisa_eg.append(xa)
SK_alisa_eg.append(pa)

    
PK_bob_rsa = []
SK_bob_rsa = []

rsabob = RSA(random_prime(2^60), random_prime(2^60))
eb, db = rsabob.generation()

PK_bob_rsa.append(eb)
PK_bob_rsa.append(rsabob.N)

SK_bob_rsa.append(db)
SK_bob_rsa.append(rsabob.N)


PK_alisa_rsa = []
SK_alisa_rsa = []

rsaalisa = RSA(random_prime(2^60), random_prime(2^60))
ea, da = rsaalisa.generation()

PK_alisa_rsa.append(ea)
PK_alisa_rsa.append(rsaalisa.N)
SK_alisa_rsa.append(da)
SK_alisa_rsa.append(rsaalisa.N)

cur_time = time.time()
ab, bb = Elgam_EncSign(elgambob, rsaalisa, PK_bob_eg, SK_alisa_rsa,  mes = 'test')
print(Elgam_encSign_dec(elgambob, rsabob, SK_bob_eg, PK_alisa_rsa,  ab, bb))
print(f"время выполнения: {time.time() - cur_time}")

test
время выполнения: 0.0013041496276855469
